# Co-occurrence analysis

This exercise will demonstrate how to perform co-occurrence analysis with **R** and the quanteda-package. It is shown how different significance measures can be used to extract semantic links between words. 

Change to your working directory, create a new R script, load the tm-package and define a few already known default variables.

In [ ]:
options(stringsAsFactors = FALSE)
require(quanteda)
require(magrittr)
require(dplyr)
require(data.table)

# read csv into a data.frame
textdata <- read.csv("data/data job posts.csv", header = TRUE, sep = ",", encoding = "UTF-8",quote = "\"")
textdata <- as.data.table(textdata)

english_stopwords <- readLines("resources/stopwords_en.txt", encoding = "UTF-8")

textdata %<>% filter(!duplicated(jobpost))
textdata %<>% mutate(d_id = 1:nrow(textdata))

#Build a dictionary of lemmas
lemmaData <- read.csv2("resources/baseform_en.tsv", sep="\t", header=FALSE, encoding = "UTF-8", stringsAsFactors = F)

data_corpus <- corpus(textdata$jobpost, docnames = textdata$d_id)

## Sentence detection

The separation of the text into semantic analysis units is important for co-occurrence analysis. Context windows can be for instance documents, paragraphs or sentences or neighboring words. One of the most frequently used context window is the sentence. 

Documents are decomposed into sentences. Sentences are defined as a separate (quasi-)documents in a new corpus object of the tm-package. The further application of the tm-package functions remains the same. In contrast to previous exercises, however, we now use sentences which are stored as individual documents in the body.

Important: The sentence segmentation must take place *before* the other preprocessing steps because the sentence-segmentation-model relies on intact word forms and punctuation marks.

The following code defines two functions. One selects documents, the other decomposes a given document text into sentences with the help of a probabilistic model.

In [ ]:
require(openNLP)

# Function to convert a document in a vector of sentences
convert_text_to_sentences <- function(text, lang = "en", SentModel = "resources/en-sent.bin") {
  
  # Calculate sentenve boundaries as annotation with Apache OpenNLP Maxent-sentence-detector.
  sentence_token_annotator <- Maxent_Sent_Token_Annotator(language = lang, model = SentModel)
  
  # Convert text to NLP string
  text <- NLP::as.String(text)
  
  # Annotate the sentence boundaries
  sentenceBoundaries <- NLP::annotate(text, sentence_token_annotator)
  
  # Select sentences as rows of a new matrix
  sentences <- text[sentenceBoundaries]
  
  # return the sentences
  return(sentences)
}

In [ ]:
# Function to convert a corpus of documents into a corpus of single sentences from documents
reshape_corpus <- function(currentCorpus, ...) {
  
  # Extraction of all sentences from the corpus as a list
  text <- lapply(currentCorpus, as.character)
  
  # convert the text
  pb <- txtProgressBar(min=0, max=length(text))
  i <- 0
  docs <- lapply(text, FUN=function(x){
    i <<- i + 1
    setTxtProgressBar(pb, i)
    convert_text_to_sentences(x)
  }, ...)
  close(pb)
  
  docs <- as.vector(unlist(docs))
  
  return(docs)
}

We apply the function `reshape_corpus` on our corpus of job posts to receive a corpus of sentences.

In [ ]:
# original corpus length and its first document
length(data_corpus$documents$texts)
substr(as.character(data_corpus$documents$texts[1]), 0, 200)
# reshape into sentences
sentences <- data.table(text=reshape_corpus(data_corpus$documents$texts))

sentences %<>% mutate(s_id = 1:nrow(sentences))

sentences_corpus <- corpus(sentences$text, docnames = sentences$s_id)

save(sentences_corpus,file="sentence_corpus.Rdata")


In [ ]:
# Preprocessing chain

#IF JAVA DID NOT WORK AND FOR TIME PURPOSE
#load(file="sentence_corpus.Rdata")

# reshaped corpus length and its first 'document'
length(sentences_corpus$documents$texts)
as.character(sentences_corpus$documents$texts[1])
as.character(sentences_corpus$documents$texts[2])

# Create a DTM (may take a while)
data_dfm_entries <- sentences_corpus %>% tokens() %>%
  tokens(remove_punct = TRUE, remove_numbers = TRUE, remove_symbols = TRUE) %>% tokens_tolower() %>% 
  tokens_replace(., lemmaData$V1, lemmaData$V2) %>%
  tokens_ngrams(1) %>% tokens_remove(pattern = stopwords()) %>% dfm()


data_dfm_entries_sub <- data_dfm_entries %>%
  dfm_select(pattern = "[a-z]", valuetype = "regex", selection = 'keep')

colnames(data_dfm_entries_sub) <- colnames(data_dfm_entries_sub) %>% stringi::stri_replace_all_regex("[^_a-z]", "") 

DTM <- dfm_compress(data_dfm_entries_sub, "features")
# Show some information
DTM

Again, we create a document-term-matrix. Only word forms which occur at least 10 times should be taken into account. An upper limit is not set (`Inf` = infinite).

Additionally, we are interested in the joint occurrence of words in a sentence. For this, we do not need the exact count of how often the terms occur, but only the information whether they occur together or not. This can be encoded in a binary document-term-matrix. The parameter `weighting` in the control options calls the ` weightBin` function. This writes a 1 into the DTM if the term is contained in a sentence and 0 if not.

In [ ]:
minimumFrequency <- 10
binDTM <- DTM %>% dfm_trim(min_docfreq = minimumFrequency)
#Make Binary
if (any(binDTM > 1)) {
   binDTM <- binDTM >= 1 + 0
  }

## Counting co-occurrences

The counting of the joint word occurrence is easily possible via a matrix multiplication (https://en.wikipedia.org/wiki/Matrix_multiplication) on the binary DTM. For this purpose, the transposed matrix (dimensions: nTypes x nDocs) is multiplied by the original matrix (nDocs x nTypes), which as a result encodes a term-term matrix (dimensions: nTypes x nTypes).

However, since we are working on very large matrices and the sparse matrix format (`slam`) which is used by the tm-package does not fully support the matrix multiplication, we first have to convert the `binDTM` into the format of the Matrix package which is more convenient to use.



In [ ]:
# Matrix multiplication for cooccurrence counts
coocCounts <- t(binDTM) %*% binDTM

Let's look at a snippet of the result. The matrix has `nTerms` rows and columns and is symmetric. Each cell contains the number of joint occurrences. In the diagonal, the frequencies of single occurrences of each term are encoded.

In [ ]:
as.matrix(coocCounts[202:205, 202:205])

Interprete as follows: *open* appears together *20878* times with *date* in the *236628* sentences of the SUTO addresses. *open* alone occurs *25147* times.

## Statistical significance

In order to not only count joint occurrence we have to determine their significance. Different significance-measures can be used. We need also various counts to calculate the significance of the joint occurrence of a term `i` (`coocTerm`) with any other term `j`:
* k - Number of all context units in the corpus
* ki - Number of occurrences of `coocTerm`
* kj - Number of occurrences of comparison term j
* kij - Number of joint occurrences of `coocTerm` and j

These quantities can be calculated for any term `coocTerm` as follows:

In [ ]:
coocTerm <- "work"
k <- nrow(binDTM)
ki <- sum(binDTM[, coocTerm])
kj <- colSums(binDTM)
names(kj) <- colnames(binDTM)
kij <- coocCounts[coocTerm, ]

An implementation in *R* for Mutual Information, Dice, and Log-Likelihood may look like this. At the end of each formula, the result is sorted so that the most significant co-occurrences are at the first ranks of the list.

In [ ]:
########## MI: log(k*kij / (ki * kj) ########
mutualInformationSig <- log(k * kij / (ki * kj))
mutualInformationSig <- mutualInformationSig[order(mutualInformationSig, decreasing = TRUE)]

########## DICE: 2 X&Y / X + Y ##############
dicesig <- 2 * kij / (ki + kj)
dicesig <- dicesig[order(dicesig, decreasing=TRUE)]
  
########## Log Likelihood ###################
logsig <- 2 * ((k * log(k)) - (ki * log(ki)) - (kj * log(kj)) + (kij * log(kij)) 
          + (k - ki - kj + kij) * log(k - ki - kj + kij) 
          + (ki - kij) * log(ki - kij) + (kj - kij) * log(kj - kij) 
          - (k - ki) * log(k - ki) - (k - kj) * log(k - kj))
logsig <- logsig[order(logsig, decreasing=T)]

The result of the four variants for the statistical extraction of co-occurrence terms is shown in a data frame below. It can be seen that frequency is a bad indicator of meaning constitution. Mutual information emphasizes rather rare events in the data. Dice and Log-likelihood yield very well interpretable contexts.

In [ ]:
# Put all significance statistics in one Data-Frame
resultOverView <- data.frame(
  names(sort(kij, decreasing=T)[1:10]), sort(kij, decreasing=T)[1:10],
  names(mutualInformationSig[1:10]), mutualInformationSig[1:10], 
  names(dicesig[1:10]), dicesig[1:10], 
  names(logsig[1:10]), logsig[1:10],
  row.names = NULL)
colnames(resultOverView) <- c("Freq-terms", "Freq", "MI-terms", "MI", "Dice-Terms", "Dice", "LL-Terms", "LL")
print(resultOverView)

## Visualization of co-occurrence

In the following, we create a network visualization of significant co-occurrences. 

For this, we provide the calculation of the co-occurrence significance measures, which we have just introduced, as single function in the file `calculateCoocStatistics.R`.  This function can be imported into the current R-Session with the `source` command.


In [ ]:
# Read in the source code for the co-occurrence calculation
source("calculateCoocStatistics.R")
# Definition of a parameter for the representation of the co-occurrences of a concept
numberOfCoocs <- 15
# Determination of the term of which co-competitors are to be measured.
coocTerm <- "experience"

We use the imported function `calculateCoocStatistics` to calculate the co-occurrences for the target term *"california"*.

In [ ]:
coocs <- calculateCoocStatistics(coocTerm, binDTM, measure="LOGLIK")
# Display the numberOfCoocs main terms
print(coocs[1:numberOfCoocs])

To acquire an extended semantic environment of the target term, 'secondary co-occurrence' terms can be computed for each co-occurrence term of the target term. This results in a graph that can be visualized with special layout algorithms (e.g. Force Directed Graph). 

Network graphs can be evaluated and visualized in R with the `igraph`-package. Any graph object can be created from a three-column data-frame. Each row in that data-frame is a triple. Each triple encodes an edge-information of two nodes (source, sink) and an edge-weight value. 

For a term co-occurrence network, each triple consists of the target word, a co-occurring word and the significance of their joint occurrence. We denote the values with *from, to, sig*. 

In [ ]:
resultGraph <- data.frame(from = character(), to = character(), sig = numeric(0))

The process of gathering the network for the target term runs in two steps. First, we obtain all significant co-occurrence terms for the target term. Second, we obtain all co-occurrences of the co-occurrence terms from step one.

Intermediate results for each term are stored as temporary triples named `tmpGraph`. With the `rbind` command ("row bind", used for concatenation of data-frames) all `tmpGraph` are appended to the complete network object stored in `resultGraph`.

In [ ]:
# The structure of the temporary graph object is equal to that of the resultGraph
tmpGraph <- data.frame(from = character(), to = character(), sig = numeric(0))

# Fill the data.frame to produce the correct number of lines
tmpGraph[1:numberOfCoocs, 3] <- coocs[1:numberOfCoocs]
# Entry of the search word into the first column in all lines
tmpGraph[, 1] <- coocTerm
# Entry of the co-occurrences into the second column of the respective line
tmpGraph[, 2] <- names(coocs)[1:numberOfCoocs]
# Set the significances
tmpGraph[, 3] <- coocs[1:numberOfCoocs]

# Attach the triples to resultGraph
resultGraph <- rbind(resultGraph, tmpGraph)

# Iteration over the most significant numberOfCoocs co-occurrences of the search term
for (i in 1:numberOfCoocs){
  
  # Calling up the co-occurrence calculation for term i from the search words co-occurrences
  newCoocTerm <- names(coocs)[i]
  coocs2 <- calculateCoocStatistics(newCoocTerm, binDTM, measure="LOGLIK")
  
  # print the co-occurrences
  coocs2[1:10]
  
  # Structure of the temporary graph object
  tmpGraph <- data.frame(from = character(), to = character(), sig = numeric(0))
  tmpGraph[1:numberOfCoocs, 3] <- coocs2[1:numberOfCoocs]
  tmpGraph[, 1] <- newCoocTerm
  tmpGraph[, 2] <- names(coocs2)[1:numberOfCoocs]
  tmpGraph[, 3] <- coocs2[1:numberOfCoocs]
  
  #Append the result to the result graph
  resultGraph <- rbind(resultGraph, tmpGraph[2:length(tmpGraph[, 1]), ])
}

As a result, `resultGraph` now contains all `numberOfCoocs * numberOfCoocs` edges of a term co-occurrence network.

In [ ]:
# Sample of some examples from resultGraph
resultGraph[sample(nrow(resultGraph), 6), ]

The package iGraph offers multiple graph visualizations for graph objects. Graph objects can be created from triple lists, such as those we just generated. In the next step we load the package iGraph and create a visualization of all nodes and edges from the object `resultGraph`.

In [ ]:
require(igraph)

# Set the graph and type. In this case, "F" means "Force Directed"
graphNetwork <- graph.data.frame(resultGraph, directed = F)

# Identification of all nodes with less than 2 edges
graphVs <- V(graphNetwork)[degree(graphNetwork) < 2]
# These edges are removed from the graph
graphNetwork <- delete.vertices(graphNetwork, graphVs) 

# Assign colors to edges and nodes (searchterm blue, rest orange)
V(graphNetwork)$color <- ifelse(V(graphNetwork)$name == coocTerm, 'cornflowerblue', 'orange') 

# Edges with a significance of at least 50% of the maximum sig- nificance in the graph are drawn in orange
halfMaxSig <- max(E(graphNetwork)$sig) * 0.5
E(graphNetwork)$color <- ifelse(E(graphNetwork)$sig > halfMaxSig, "coral", "azure3")

# Disable edges with radius
E(graphNetwork)$curved <- 0 
# Size the nodes by their degree of networking
V(graphNetwork)$size <- log(degree(graphNetwork)) * 5

# All nodes must be assigned a standard minimum-size
V(graphNetwork)$size[V(graphNetwork)$size < 5] <- 3 

# edge thickness
E(graphNetwork)$width <- 2

# Define the frame and spacing for the plot
par(mai=c(0,0,1,0)) 

# Finaler Plot
plot(graphNetwork,				
     layout = layout.fruchterman.reingold,	# Force Directed Layout 
     main = paste(coocTerm, ' Graph'),
     vertex.label.family = "sans",
     vertex.label.cex = 0.8,
     vertex.shape = "circle",
     vertex.label.dist = 0.5,			# Labels of the nodes moved slightly
     vertex.frame.color = 'darkolivegreen',
     vertex.label.color = 'black',		# Color of node names
     vertex.label.font = 2,			# Font of node names
     vertex.label = V(graphNetwork)$name,		# node names
     vertex.label.cex = 1 # font size of node names 
)